### loading raw json file into dataframe df

In [2]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b30a44ae-dcd8-4fb8-af91-64f25bc21876)

### extracting actual values we need

In [3]:
df = df.select("value")

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 5, Finished, Available, Finished)

In [4]:
display(df)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ece2464d-2d64-4128-b3be-93a97b21437a)

### change from single row to multiple using explode function

In [5]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["value"].alias("col")))

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 7, Finished, Available, Finished)

In [6]:
display(df_exploded)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c54b724d-130c-443f-9d33-9848796f55ef)

In [7]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 9, Finished, Available, Finished)

### checking the json_list

In [8]:
print(json_list)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 10, Finished, Available, Finished)

['{"col":{"about":[{"name":"Arsenal F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/8b3b4135-cb8e-ded8-10ce-8f130d64b863"},{"name":"Riccardo Calafiori","readLink":"https://api.bing.microsoft.com/api/v7/entities/49f29b7c-9e4d-9c42-9c72-bfde27d2fe38"},{"name":"Nico Williams","readLink":"https://api.bing.microsoft.com/api/v7/entities/d5ea2495-e221-af84-25b3-f794c796ed05"},{"name":"Bologna F.C. 1909","readLink":"https://api.bing.microsoft.com/api/v7/entities/1f91cf4d-6c83-6e32-55aa-39774443c42f"},{"name":"Euro","readLink":"https://api.bing.microsoft.com/api/v7/entities/b3e7acef-3980-85b9-ba45-29b117eff778"},{"name":"Brentford","readLink":"https://api.bing.microsoft.com/api/v7/entities/6ebc31f5-36a7-3b3d-18b1-e6be3d21fd3e"},{"name":"David Raya","readLink":"https://api.bing.microsoft.com/api/v7/entities/36c81142-7e92-345e-7287-5f4f5e704b75"},{"name":"England","readLink":"https://api.bing.microsoft.com/api/v7/entities/280d39e8-7217-6863-6980-a8c20c211c89"},{"name":"UEFA Euro 

### checking value at 0

In [9]:
print(json_list[0])

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 11, Finished, Available, Finished)

{"col":{"about":[{"name":"Arsenal F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/8b3b4135-cb8e-ded8-10ce-8f130d64b863"},{"name":"Riccardo Calafiori","readLink":"https://api.bing.microsoft.com/api/v7/entities/49f29b7c-9e4d-9c42-9c72-bfde27d2fe38"},{"name":"Nico Williams","readLink":"https://api.bing.microsoft.com/api/v7/entities/d5ea2495-e221-af84-25b3-f794c796ed05"},{"name":"Bologna F.C. 1909","readLink":"https://api.bing.microsoft.com/api/v7/entities/1f91cf4d-6c83-6e32-55aa-39774443c42f"},{"name":"Euro","readLink":"https://api.bing.microsoft.com/api/v7/entities/b3e7acef-3980-85b9-ba45-29b117eff778"},{"name":"Brentford","readLink":"https://api.bing.microsoft.com/api/v7/entities/6ebc31f5-36a7-3b3d-18b1-e6be3d21fd3e"},{"name":"David Raya","readLink":"https://api.bing.microsoft.com/api/v7/entities/36c81142-7e92-345e-7287-5f4f5e704b75"},{"name":"England","readLink":"https://api.bing.microsoft.com/api/v7/entities/280d39e8-7217-6863-6980-a8c20c211c89"},{"name":"UEFA Euro 20

### trnasforming data from string to dictionary

In [10]:
import json

news_json = json.loads(json_list[0])

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 12, Finished, Available, Finished)

In [11]:
print(news_json['col']['name'])
print(news_json['col']['description'])
print(news_json['col']['category'])
print(news_json['col']['url'])
print(news_json['col']['image']['thumbnail']['contentUrl'])
print(news_json['col']['provider'][0]['name'])
print(news_json['col']['datePublished'])


StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 13, Finished, Available, Finished)

Arsenal transfer news LIVE: Riccardo Calafiori ‘personal terms AGREED’ for £27m switch, Nico Williams latest updates
ARSENAL are closing in on Riccardo Calafiori – with personal reportedly terms agreed for Bologna’s £27m defender. Meanwhile, Arsenal are also interested in Spanish youngster Nico
Sports
https://www.thesun.co.uk/sport/27840895/arsenal-transfer-news-live-riccardo-calafiori-updates/
https://www.bing.com/th?id=OVFT.J19yyRNtGRQaOc5xtdY0sC&pid=News
The Sun
2024-07-08T19:02:00.0000000Z


title,
description,
category,
url,
image,
provider,
datePublished

### transforming all the data using for loop, if image and category is missing data is ignored

In [12]:
title = []
description =[]
category =[]
url =[]
image =[]
provider =[]
datePublished =[]

#process each JSON object in the list
for json_str in json_list:
    try:
        #parse the JSON string into dictionary
        article = json.loads(json_str)

        if article["col"].get("category") and article["col"].get("image",{}).get("thumbnail",{}).get("contentUrl"):
            #extract information from the dictionary
            title.append(article["col"]["name"])
            description.append(article["col"]["description"])
            category.append(article["col"]["category"])
            url.append(article["col"]["url"])
            image.append(article["col"]["image"]["thumbnail"]["contentUrl"])
            provider.append(article["col"]["provider"][0]["name"])
            datePublished.append(article["col"]["datePublished"])

        

    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 14, Finished, Available, Finished)

In [13]:
title

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 15, Finished, Available, Finished)

['Arsenal transfer news LIVE: Riccardo Calafiori ‘personal terms AGREED’ for £27m switch, Nico Williams latest updates',
 'Man Utd transfer news LIVE: United ‘AGREE terms with Joshua Zirkzee’, £35m star set to LEAVE, Chiesa BOOST – updates',
 "NYC mayor faces bipartisan blowback over new round of migrant debit cards: 'The gravy train’ must stop",
 'Manchester United transfer news live Joshua Zirkzee update as Mason Greenwood exit talks advanced',
 'Transfer news live: Man Utd finalising Zirkzee move; Premier League, MLS latest updates and more',
 'Spain XI vs France: Confirmed Euro 2024 team news, predicted lineup and injury latest for semi-final',
 'A billionaire is boosting a major defamation lawsuit against Fox News',
 'Breaking News LIVE: Hemant Soren Govt Wins Trust Vote in Jharkhand Assembly',
 "Best New Shows and Movies to Stream for the Week of July 8: 'Descendents,' 'The Bachelorette' and More",
 'Breaking News Live Updates: BSP chief Mayawati pays homage to Tamil Nadu Preside

### coverting the list to dataframe

In [14]:
from pyspark.sql.types import StructType, StructField, StringType

#combine the lists
data = list(zip(title, description, category, url, image, provider, datePublished))

#define schema
schema = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])
df_cleaned = spark.createDataFrame(data, schema= schema)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 16, Finished, Available, Finished)

In [15]:
display(df_cleaned)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ade8c26a-99af-44fa-8973-0de948895450)

### trnasforming "datePublished" column

In [16]:
from pyspark.sql.functions import to_date, date_format
df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-mm-yyyy"))

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 18, Finished, Available, Finished)

In [17]:
display(df_cleaned_final)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 83b50914-e369-4a19-b570-08872f277d46)

Writing the final dataframe to the bing_lake_DB in a Delta format

In [22]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_latest_news'
    
    df_cleaned_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:
    print("Table Already exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_df_cleaned_final AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title <> target_table.title OR
            source_view.description <> target_table.description OR
            source_view.category <> target_table.category OR
            source_view.image <> target_table.image OR
            source_view.provider <> target_table.provider OR
            source_view.datePublished <> target_table.datePublished
        )
        THEN UPDATE SET *
        WHEN NOT MATCHED THEN INSERT *
    """)

StatementMeta(, 1657c998-8f32-468d-9b27-0c935d257f52, 24, Finished, Available, Finished)

Table Already exists
